# Sentiment Analysis Part 2

_Natural Langauge Processing Nanodegree Program_

---



## Step 5: Switching gears - RNNs

We just saw how the task of sentiment analysis can be solved via a traditional machine learning approach: BoW + a nonlinear classifier. We now switch gears and use Recurrent Neural Networks, and in particular LSTMs, to perform sentiment analysis in Keras. Conveniently, Keras has a built-in [IMDb movie reviews dataset](https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification) that we can use, with the same vocabulary size.

In [15]:
from keras.datasets import imdb  # import the built-in imdb dataset in Keras

# Set the vocabulary size
vocabulary_size = 5000

# Load in training and test data (note the difference in convention compared to scikit-learn)
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocabulary_size)
print("Loaded dataset with {} training samples, {} test samples".format(len(X_train), len(X_test)))

Loaded dataset with 25000 training samples, 25000 test samples


In [16]:
# Inspect a sample review and its label
print("--- Review ---")
print(X_train[7])
print("--- Label ---")
print(y_train[7])

--- Review ---
[1, 13, 197, 14, 16, 6, 389, 96, 8, 1142, 58, 23, 6, 99, 893, 1506, 2480, 1266, 11, 4, 945, 2, 750, 5, 149, 6, 641, 2287, 212, 4, 114, 9, 4274, 21, 4, 414, 9, 1920, 5, 4, 105, 26, 1451, 60, 4, 73, 2, 2, 1435, 455, 137, 49, 203, 30, 685, 54, 36, 923, 14, 9, 24, 1014, 213, 241, 2960, 2, 13, 197, 12, 16, 3170, 15, 2292, 1602, 9, 131, 1314, 11, 1140, 7, 4, 405, 111, 7, 178, 28, 2071, 8, 119, 10, 10, 14, 16, 4, 91, 474, 1498, 33, 31, 7, 2, 1290, 11, 153, 3328, 13, 135, 6, 2068, 137, 207, 115, 77, 1555, 19, 2, 2, 11, 14, 59, 1319, 8, 1163, 180, 41, 1279, 1459, 5, 4584, 208, 83, 6, 856, 21, 3649, 185, 255, 10, 10, 14, 203, 24, 30, 4, 2, 2, 7, 27, 611, 21, 12, 16, 2, 74, 2113, 2899, 2, 5, 53, 221, 74, 1832, 6, 87, 212, 8, 140, 67, 19, 369]
--- Label ---
1


Notice that the label is an integer (0 for negative, 1 for positive), and the review itself is stored as a sequence of integers. These are word IDs that have been preassigned to individual words. To map them back to the original words, you can use the dictionary returned by `imdb.get_word_index()`.

In [17]:
# Map word IDs back to words
word2id = imdb.get_word_index()
id2word = {i: word for word, i in word2id.items()}
print("--- Review (with words) ---")
print([id2word.get(i, " ") for i in X_train[7]])
print("--- Label ---")
print(y_train[7])

--- Review (with words) ---
['the', 'was', 'between', 'as', 'with', 'is', 'small', 'too', 'in', 'zombies', 'my', 'are', 'is', 'movies', 'baby', 'porn', 'mainstream', 'week', 'this', 'of', 'creepy', 'and', 'dull', 'to', "doesn't", 'is', 'usual', 'involves', 'must', 'of', 'little', 'it', 'eighties', 'not', 'of', 'finally', 'it', 'mine', 'to', 'of', 'films', 'he', 'falling', 'which', 'of', 'much', 'and', 'and', 'emotions', 'direction', 'go', 'good', 'action', 'at', 'due', 'no', 'from', 'society', 'as', 'it', 'his', 'expecting', 'come', 'am', 'essential', 'and', 'was', 'between', 'that', 'with', 'lying', 'for', 'k', 'quick', 'it', 'these', 'key', 'this', 'band', 'br', 'of', 'gives', 'plot', 'br', 'want', 'one', 'boat', 'in', 'did', 'i', 'i', 'as', 'with', 'of', 'its', 'final', 'mediocre', 'they', 'by', 'br', 'and', 'beautifully', 'this', 'actors', 'trade', 'was', 'why', 'is', 'grown', 'go', 'always', 'best', 'will', 'ugly', 'film', 'and', 'and', 'this', 'as', 'would', 'laughable', 'in', 'a

In [12]:
min_words, max_words = len(X_train),0
for words in X_train:
    if len(words)<min_words:
        min_words = len(words)
    if len(words)>max_words:
        max_words = len(words)
print(min_words, max_words)

11 2494


Unlike our Bag-of-Words approach, where we simply summarized the counts of each word in a document, this representation essentially retains the entire sequence of words (minus punctuation, stopwords, etc.). This is critical for RNNs to function. But it also means that now the features can be of different lengths!

#### Question: Variable length reviews

What is the maximum review length (in terms of number of words) in the training set? What is the minimum?

#### Answer:

minimum: 11
maximum: 2494


### TODO: Pad sequences

In order to feed this data into your RNN, all input documents must have the same length. Let's limit the maximum review length to `max_words` by truncating longer reviews and padding shorter reviews with a null value (0). You can accomplish this easily using the [`pad_sequences()`](https://keras.io/preprocessing/sequence/#pad_sequences) function in Keras. For now, set `max_words` to 500.

In [18]:
from keras.preprocessing import sequence

# Set the maximum number of words per document (for both training and testing)
max_words = 500

# TODO: Pad sequences in X_train and X_test
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

### TODO: Design an RNN model for sentiment analysis

Build your model architecture in the code cell below. We have imported some layers from Keras that you might need but feel free to use any other layers / transformations you like.

Remember that your input is a sequence of words (technically, integer word IDs) of maximum length = `max_words`, and your output is a binary sentiment label (0 or 1).

In [20]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

# TODO: Design your model
embedding_size = 32
model = Sequential()
model.add(Embedding(vocabulary_size, embedding_size, input_length=max_words))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.1))
model.add(Dense(1,activation='sigmoid'))

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 500, 32)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


#### Question: Architecture and parameters

Briefly describe your neural net architecture. How many model parameters does it have that need to be trained?

#### Answer:

The RNN-LSTM model is structured as three-layered. At first, Embedding layer imports inputs of 160,000 params (500*32) with Dropout late of 0.2. Second layer is the core of the morel, LSTM, which takes additional 53,000 params. Finally, dense layer for output is added.

### TODO: Train and evaluate your model

Now you are ready to train your model. In Keras world, you first need to _compile_ your model by specifying the loss function and optimizer you want to use while training, as well as any evaluation metrics you'd like to measure. Specify the approprate parameters, including at least one metric `'accuracy'`.

In [21]:
# TODO: Compile your model, specifying a loss function, optimizer, and metrics
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

Once compiled, you can kick off the training process. There are two important training parameters that you have to specify - **batch size** and **number of training epochs**, which together with your model architecture determine the total training time.

Training may take a while, so grab a cup of coffee, or better, go for a hike! If possible, consider using a GPU, as a single training run can take several hours on a CPU.

> **Tip**: You can split off a small portion of the training set to be used for validation during training. This will help monitor the training process and identify potential overfitting. You can supply a validation set to `model.fit()` using its `validation_data` parameter, or just specify `validation_split` - a fraction of the training data for Keras to set aside for this purpose (typically 5-10%). Validation metrics are evaluated once at the end of each epoch.

In [22]:
# TODO: Specify training parameters: batch size and number of epochs
batch_size = 64
num_epochs = 10

# TODO(optional): Reserve/specify some training data for validation (not to be used for training)
X_valid, y_valid = X_train[:batch_size], y_train[:batch_size]
X_train2, y_train2 = X_train[batch_size:], y_train[batch_size:]

# TODO: Train your model
model.fit(X_train2, y_train2, validation_data = (X_valid,y_valid), batch_size=batch_size, epochs=num_epochs)

Train on 24936 samples, validate on 64 samples
Epoch 1/10
24936/24936 [==============================] - 550s 22ms/step - loss: 0.4692 - acc: 0.7688 - val_loss: 0.4518 - val_acc: 0.8594
Epoch 2/10
24936/24936 [==============================] - 553s 22ms/step - loss: 0.3491 - acc: 0.8536 - val_loss: 0.2955 - val_acc: 0.8750
Epoch 3/10
24936/24936 [==============================] - 547s 22ms/step - loss: 0.2515 - acc: 0.9014 - val_loss: 0.3147 - val_acc: 0.9062
Epoch 4/10
24936/24936 [==============================] - 542s 22ms/step - loss: 0.2262 - acc: 0.9141 - val_loss: 0.4473 - val_acc: 0.8438
Epoch 5/10
24936/24936 [==============================] - 540s 22ms/step - loss: 0.2025 - acc: 0.9232 - val_loss: 0.3346 - val_acc: 0.8594
Epoch 6/10
24936/24936 [==============================] - 551s 22ms/step - loss: 0.1774 - acc: 0.9329 - val_loss: 0.2848 - val_acc: 0.8438
Epoch 7/10
24936/24936 [==============================] - 555s 22ms/step - loss: 0.1697 - acc: 0.9362 - val_loss: 0.308

In [26]:
import os
cache_dir = os.path.join("cache", "sentiment_analysis")  # where to store cache files
os.makedirs(cache_dir, exist_ok=True)

# Save your model, so that you can quickly load it in future (and perhaps resume training)
model_file = "rnn_model.h5"  # HDF5 file
model.save(os.path.join(cache_dir, model_file))

# Later you can load it using keras.models.load_model()
#from keras.models import load_model
#model = load_model(os.path.join(cache_dir, model_file))

Once you have trained your model, it's time to see how well it performs on unseen test data.

In [27]:
# Evaluate your model on the test set
scores = model.evaluate(X_test, y_test, verbose=0)  # returns loss and other metrics specified in model.compile()
print("Test accuracy:", scores[1])  # scores[1] should correspond to accuracy if you passed in metrics=['accuracy']

Test accuracy: 0.86708


#### Question: Comparing RNNs and Traditional Methods

How well does your RNN model perform compared to the BoW + Gradient-Boosted Decision Trees?

#### Answer:

BoW: 0.81
RNN: 0.87

## Extensions

There are several ways in which you can build upon this notebook. Each comes with its set of challenges, but can be a rewarding experience.

- The first thing is to try and improve the accuracy of your model by experimenting with different architectures, layers and parameters. How good can you get without taking prohibitively long to train? How do you prevent overfitting?

- Then, you may want to deploy your model as a mobile app or web service. What do you need to do in order to package your model for such deployment? How would you accept a new review, convert it into a form suitable for your model, and perform the actual prediction? (Note that the same environment you used during training may not be available.)

- One simplification we made in this notebook is to limit the task to binary classification. The dataset actually includes a more fine-grained review rating that is indicated in each review's filename (which is of the form `<[id]_[rating].txt>` where `[id]` is a unique identifier and `[rating]` is on a scale of 1-10; note that neutral reviews > 4 or < 7 have been excluded). How would you modify the notebook to perform regression on the review ratings? In what situations is regression more useful than classification, and vice-versa?

Whatever direction you take, make sure to share your results and learnings with your peers, through blogs, discussions and participating in online competitions. This is also a great way to become more visible to potential employers!